# Processing MCSL summer swimming result

In [115]:
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd
import re

In [116]:
#file/folder
htmlbasepath = "./data/2024/week1"
filename_html = 'MS-RC.html'
filename_output = filename_html.replace('.html','.csv')

htmlpath = os.path.join(htmlbasepath, f'{filename_html}')
# print(path)

In [117]:
htmlfile_list = htmlbasepath.split('/')
year = htmlfile_list[2]
week = htmlfile_list[3]

In [118]:
with open(htmlpath, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')

# print(soup.prettify())

In [119]:
# Find all <h4> tags (event names)
event_tags = soup.find_all('h4')

# Initialize an empty list to store event data
events = []

# Loop through each event tag
for event_tag in event_tags:
    event_name = event_tag.text.strip()

    # Find the corresponding table for each event
    table = event_tag.find_next('table')
    rows = table.find_all('tr')

    # Initialize an empty list for this event's data
    event_data = []

 # Loop through rows (skip the header row)
    for row in rows[1:]:
        cells = row.find_all('td')
        if cells:
            position, name, seed_time, final_time, points = [cell.text.strip() for cell in cells]
            event_data.append({
                'position': position,
                'name': name,
                'seed_time': seed_time,
                'final_time': final_time,
                'points': points
            })

    # Append the event data to the list
    events.append({'event_name': event_name, 'data': event_data})

# Create new list for dataframe
data_list=[]
for event in events:
    event_name = event['event_name']
    # print(f"Event Name: {event['event_name']}"
    for data in event['data']:
        # print(f"year: {year}, week: {week}, event: {event_name}, rank: {data['position']}, swimmer: {data['name']}, seed: {data['seed_time']}, final: {data['final_time']}, point: {data['points']}")
        data_list.append({
                    'year': year,
                    'week': week,
                    'event': event_name,
                    'rank': data['position'],
                    'swimmer': data['name'],
                    'seed': data['seed_time'],
                    'final': data['final_time'],
                    'point': data['points']
                })


In [120]:
# df = pd.DataFrame(individual_record, columns =['year', 'week', 'event', 'rank', 'swimmer', 'seed', 'final']) 
df = pd.DataFrame(data_list)

In [113]:
df.to_csv('test.csv')

In [121]:
df.head()

,year,week,event,rank,swimmer,seed,final,point
0,2024,week1,Event 1 - Male 12&U 100M Medley,1,"Ahuja, Riaan (12)(MS)",NT,1:09.62,6
1,2024,week1,Event 1 - Male 12&U 100M Medley,2,"Kimpel, Ross (12)(RC)",1:25.45,1:24.03,4
2,2024,week1,Event 1 - Male 12&U 100M Medley,3,"Mendizabal, Kai (11)(RC)",1:27.06,1:25.62,3
3,2024,week1,Event 1 - Male 12&U 100M Medley,4,"Carare, Eli C (12)(RC)",1:32.69,1:28.69,2
4,2024,week1,Event 1 - Male 12&U 100M Medley,5,"Rosenbaum, Ryan (11)(MS)",1:36.28,1:38.29,1


In [122]:
#special handling for swimmer with () on its name
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Dan)', '- Dan'))
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Ben)', '- Ben'))
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Jojo)', '- Jojo'))
df['swimmer'] = df['swimmer'].apply(lambda x:x.replace('(Jorie)', '- Jorie'))


#splitting swimmer colunn into 3
df[['swimmer_name', 'swimmer_age', 'swimmer_team']] = df['swimmer'].str.split(pat='(', expand=True, n=2)

# #dropping progoma; swimmer column
# del df['swimmer']

In [126]:
df['swimmer_age']

0                          12)
1                          12)
2                          11)
3                          12)
4                          11)
                ...           
334        11), Goode, Nathan 
335          11), Goode, Emma 
336    14), Chery, Jennifer H 
337       9), Castro, Elise A 
338    14), Saslowsky, Vivian 
Name: swimmer_age, Length: 339, dtype: object

In [125]:
# #replace function
# def replace_enhanced(x):
#     if x is None:
#         return(x)
#     else:
#         x.replace(')','')
#         return(x)
    
#remove ')' from age and team
df['swimmer_age'] = df['swimmer_age'].apply(lambda x:x.replace(')', ''))
df['swimmer_team'] = df['swimmer_team'].apply(lambda x:x.replace(')', ''))


AttributeError: 'NoneType' object has no attribute 'replace'

In [ ]:
# #add seconds for seed and final
# x = '34.88'
# x_list = x.split(':')

# time_list = x.split(':')
# seconds = float(time_list[0])*60 + float(time_list[1])
# print(seconds)


def x_to_seconds(x):
    x = x.replace('X', '') #replacing exibition time
    x_list = x.split(':')
    if x in ('NT', 'NS', 'DQ', 'DNF'):  # ignore NT, NS, DQ, and DNF
        return(x)
    elif len(x_list) == 1: #only seconds no minutes
         return(float(x_list[0]))
    else:
        return(float(x_list[0])*60 + float(x_list[1]))
# x_to_seconds(x)


df['seed_seconds'] = df['seed'].apply(lambda x:x_to_seconds(x))
df['final_seconds'] = df['final'].apply(lambda x:x_to_seconds(x))



In [ ]:
#get event number
df['event_no'] = df['event'].apply(lambda x:x.split('-')[0].replace('Event ', ''))
#remove relay events (27,28,49,50) , i am using space at the end cause data has it
df = df[~df['event_no'].isin(['27 ','28 ' ,'49 ','50 '])]

In [ ]:
df.tail(10).T

,334,335,336,337,338,339,340,341,342,343
year,2024,2024,2024,2024,2024,2024,2024,2024,2024,2024
week,week1,week1,week1,week1,week1,week1,week1,week1,week1,week1
event,Event 47 - Male 15-18 50M Butterfly,Event 47 - Male 15-18 50M Butterfly,Event 47 - Male 15-18 50M Butterfly,Event 47 - Male 15-18 50M Butterfly,Event 48 - Female 15-18 50M Butterfly,Event 48 - Female 15-18 50M Butterfly,Event 48 - Female 15-18 50M Butterfly,Event 48 - Female 15-18 50M Butterfly,Event 48 - Female 15-18 50M Butterfly,Event 48 - Female 15-18 50M Butterfly
rank,3,4,5,6,1,2,3,4,5,6
swimmer,"Cobb, Jason A (15)(SB)","Lei, Jonathan (16)(SB)","Kaplan, Caleb (15)(RF)","Armstrong, Christian (16)(RF)","Harms, Sofia M (15)(RF)","Shad, Sophia (18)(SB)","Chen, Elizabeth (15)(SB)","Rampulla, Maya (15)(RF)","Woitach, Olivia (17)(RF)","Lai, Valerie (15)(SB)"
seed,NT,31.25,NT,31.63,32.28,32.77,32.94,36.06,NT,NT
final,31.16,31.28,31.65,32.13,31.60,33.23,33.48,34.43,39.82,47.81
point,3,2,1,0,6,4,3,2,1,0
swimmer_name,"Cobb, Jason A","Lei, Jonathan","Kaplan, Caleb","Armstrong, Christian","Harms, Sofia M","Shad, Sophia","Chen, Elizabeth","Rampulla, Maya","Woitach, Olivia","Lai, Valerie"
swimmer_age,15,16,15,16,15,18,15,15,17,15


In [ ]:
df.tail()

,year,week,event,rank,swimmer,seed,final,point,swimmer_name,swimmer_age,swimmer_team,seed_seconds,final_seconds,event_no
339,2024,week1,Event 48 - Female 15-18 50M Butterfly,2,"Shad, Sophia (18)(SB)",32.77,33.23,4,"Shad, Sophia",18,SB,32.77,33.23,48
340,2024,week1,Event 48 - Female 15-18 50M Butterfly,3,"Chen, Elizabeth (15)(SB)",32.94,33.48,3,"Chen, Elizabeth",15,SB,32.94,33.48,48
341,2024,week1,Event 48 - Female 15-18 50M Butterfly,4,"Rampulla, Maya (15)(RF)",36.06,34.43,2,"Rampulla, Maya",15,RF,36.06,34.43,48
342,2024,week1,Event 48 - Female 15-18 50M Butterfly,5,"Woitach, Olivia (17)(RF)",NT,39.82,1,"Woitach, Olivia",17,RF,NT,39.82,48
343,2024,week1,Event 48 - Female 15-18 50M Butterfly,6,"Lai, Valerie (15)(SB)",NT,47.81,0,"Lai, Valerie",15,SB,NT,47.81,48


In [ ]:
#export to csv with | delimited
df.to_csv(filename_output,sep='|', index=False) # Use pipe to seperate data


In [ ]:
#counting swimmer by team/age
df.groupby(['swimmer_age', 'swimmer_team'])['swimmer_name'].count()

swimmer_age  swimmer_team
10           RF              14
             SB              27
11           RF               5
             SB              20
12           RF              31
             SB              16
13           RF              15
             SB              24
14           RF              21
             SB              11
15           RF              17
             SB              14
16           RF              15
             SB               6
17           RF               4
             SB               6
18           SB              10
6            RF               1
             SB               3
7            RF               1
             SB               6
8            RF              28
             SB              21
9            RF              16
             SB               4
Name: swimmer_name, dtype: int64